## DEAP: Genetic Algorithm and Genetic Programming Tutorial

In [ ]:
%pip install deap 

In [ ]:
import random
import numpy as np

In [ ]:
from deap import base, creator, tools, algorithms

### Simple Genetic Algorithm

<b>Overall flow of a *Genetic Algorithm*:</b>

    1) Population Initialisation 
    2) Fitness computation
    3) Evolution (crossover and mutation) 
    4) Next population selection (include fitness computation)
    5) Termination check 
        5-1) yes: Done. Return the best solution found so far 
        5-2) no: go to 3)  

<b>Four main steps of DEAP</b>: 

    1) Type Definition, 
    2) Initialisation, 
    3) Operator defintion & register, 
    4) Algorithm 

#### Examples: one-max problem 

Evolve until obtaining an individual with all 1s. 

<b>1) Type defintion </b>

In [ ]:
# define the individual type for one-max problem 
creator.create("FitnessMax", base.Fitness, weights = (1.0,))
creator.create("OneMaxIndividual", list, fitness = creator.FitnessMax)

In [ ]:
# ..

<b>2) Initialisation </b>

In [ ]:
ind_size = 100
pop_size = 50

toolbox = base.Toolbox() # a contaner for tools of all sorts including initialiser 
# define intialisers and register them into toolbox
toolbox.register("attribute", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.OneMaxIndividual, toolbox.attribute, n = ind_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, n = pop_size)


<b>3) Operator defintion & register </b>

In [ ]:
# all operators that will be used during GA including the evolution operators 
def count_one(ind): # ~= fitness computation 
    return (np.sum(ind),)

toolbox.register("evaluate", count_one) # can pass the data here
toolbox.register("mate", tools.cxOnePoint) 
toolbox.register("mutate", tools.mutFlipBit, indpb = 1/ind_size) 
toolbox.register("select", tools.selTournament, tournsize = 3, fit_attr = "fitness") # fit_tarr = the name of the attribute to use, 

In [ ]:
# checking 
toolbox.evaluate

<b>4) Algorithm </b>

In [ ]:
n_gen = 50
cxRate = 1.0
mutRate = 0.2

In [ ]:
## 1) population initialisation 
pop = toolbox.population() # n = new_pop_size

In [ ]:
pop[0].fitness.values, pop[0].fitness.valid # none assigned

In [ ]:
## 2) fitness evaluation 
fitness_per_ind = toolbox.map(toolbox.evaluate, pop)
for ind,fit_v in zip(pop, fitness_per_ind):
    ind.fitness.values = fit_v

In [ ]:
## 2-2) select the best individual & store
best = tools.HallOfFame(1) # similar = ... # define for duplicate check 
print (best)
best.update(pop)
print (best[0])

In [ ]:
## logging ##
stats = tools.Statistics(lambda ind:ind.fitness.values[0])
stats.register("max", np.max, axis = 0)
stats.register("min", np.min, axis = 0)
stats.register("average", np.mean, axis = 0)

In [ ]:
stat_results = stats.compile(pop)
print (stat_results)

In [ ]:
logbook = tools.Logbook()
logbook.record(
    gen = 0, 
    max = stat_results['max'],
    min = stat_results['min'],
    mean = stat_results['average'],
)
print (logbook)

In [ ]:
## 3) evolution & 4) next population selection 
from tqdm import tqdm 
allow_dup = False #True
for gen_idx in tqdm(range(1, n_gen + 1)):
    # 3) evoluation 
    offsprings = []
    pop_copied = [toolbox.clone(ind) for ind in pop]
    while len(offsprings) < pop_size:
        offspring = toolbox.select(pop_copied, 2)
        ## crossover
        if random.random() < cxRate:
            offspring[0], offspring[1] = toolbox.mate(offspring[0], offspring[1])
            del offspring[0].fitness.values, offspring[1].fitness.values
        # muate 
        if random.random() < mutRate:
            offspring[0], = toolbox.mutate(offspring[0])
            del offspring[0].fitness.values
        if random.random() < mutRate:
            offspring[1], = toolbox.mutate(offspring[1])
            del offspring[1].fitness.values
        ind1, ind2 = offspring
        #ind1, ind2 = algorithms.varAnd(offspring, toolbox, cxRate, mutRate) 
        
        # check duplicate & include 
        if len(offsprings) == 0 or allow_dup:
            offsprings.extend([ind1, ind2])
        else:
            incl = True 
            for _ind in offsprings:
                if str(_ind).__eq__(str(ind1)): incl = False; break 
            if incl: 
                offsprings.append(ind1)
            incl = True 
            for _ind in offsprings:
                if str(_ind).__eq__(str(ind2)): incl = False; break 
            if incl: 
                offsprings.append(ind2) 
    
    # 4) next population selection 
    ind_fit_inv = [ind for ind in offsprings if not ind.fitness.valid]
    fitness_per_ind = toolbox.map(toolbox.evaluate, ind_fit_inv)
    for ind, fit_v in zip(ind_fit_inv, fitness_per_ind):
        ind.fitness.values = fit_v 
    ## 4-2) the next population selection 
    pop[:] = tools.selBest(pop + offsprings, pop_size)
    # update best
    best.update(pop)

    # logbox 
    stat_results = stats.compile(pop)
    max_v, min_v, avg_v = stat_results['max'], stat_results['min'], stat_results['average']
    logbook.record(gen = gen_idx, max = max_v, min = min_v, mean = avg_v)
    #print (logbook)
    # some termination condition: the number of unchanged cases 
    if toolbox.evaluate(best[0])[0] == ind_size:
        print (f"At Gen {gen_idx}, the optimal found")
        break 

print (logbook)
print ("\n")
print (best[0])

In [ ]:
# visulation 
import matplotlib.pyplot as plt 
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)

gen_vs, min_vs, max_vs, mean_vs = logbook.select("gen", "min", "max", 'mean')
ax.plot(gen_vs, min_vs, c = 'r', label = 'min')
ax.plot(gen_vs, max_vs, c = 'b', label = 'max')
ax.plot(gen_vs, mean_vs, c = 'g', label = 'mean')

ax.grid()
plt.legend()
plt.show()

#### Using pre-implemented evolutionary algorithms

In [ ]:
pop = toolbox.population(n = pop_size)
algorithms.eaSimple(pop, toolbox, cxRate, mutRate, ngen=n_gen, stats = stats, verbose= False)
best_found = tools.selBest(pop, 1)
print (best_found, np.sum(best_found))

#### Exercises

<b>Traveling Salesman Problem</b>

* Individual: a candidate city visiting order
* Fitness: minmise the total distance when visiting the cities sequentially as indicated in the individual
* crossover: 
    - tools.cxOrderd 
* mutation: shuffligng 
    - tools.mutShuffleIndexs, indpb = 1/n_cities 
* parent selection:
    - tools.selTournament, tournsize = 3
* next population selection: select the top pop_size best individuals 
    - tools.selBest 

In [ ]:
pop_size = 100
n_cities = 10

In [ ]:
# provided
from itertools import combinations
edges = list(combinations(np.arange(n_cities), 2))
dist_btwn_cities = {}
for c1,c2 in edges:
    dist = np.random.rand()
    dist_btwn_cities[(c1,c2)] = dist 
    dist_btwn_cities[(c2,c1)] = dist 

In [ ]:
# define types 
...

# initialisation  
# np.random.permutation with n_cities 
...

# operator 
# evaluate, mate (crossover), mutate, selction 

In [ ]:
n_gen = 50
cxRate = 1.0 
mutRate = 0.2 

# algorithm 
# 1) population initialisation 
pop = ... 
# 2) fitness evaluation 
...

# 2-2) select the best individual & store (e.g., HallofFame)
...

# logging (choice)
#stats = tools.Statistics(lambda ind: ind.fitness.values[0])
#stats.register("max", np.max, axis = 0)
#stats.register("min", np.min, axis = 0)
#stats.register("mean", np.mean, axis = 0)
#stat_results = stats.compile(pop)
#logbook = tools.Logbook()
#logbook.record(gen = 0, max = stat_results['max'], min = stat_results['min'], mean = stat_results['mean'])

## 3) evolution & 4) next population selection
for gen_idx in range(1, n_gen+1):
    # 3) evolution => can use varAnd or implement own building block 
    # crossover, mutate 
    ...

    # 4) next population selection 
    ## 4-1) fitness computation for new offsrpings 
    ...

    ## 4-2) the next population selection 
    ...

    # update the best solution (HallOfFame)
    ...
    # stat_results = stats.compile(pop) # choice
    #logbook.record(gen = gen_idx, max = stat_results['max'], min = stat_results['min'], mean = stat_results['mean'])
    #print (f"\tGen {gen_idx}, best: {best[0]}, fitness: {best[0].fitness.values[0]}") 
    #print (logbook)

### Simple Genetic Programming

In [ ]:
from deap import gp 

#### Examples: symbolic regression 

Automatically generate (or search) a model/experssion that best-fits a given dataset

* target: $f(x) = x^4 + x^3 + x^2 + x$
* dataset: $x = -1.0 ~ 1.0$ with step 0.05, $y = \{f(x)| \in x\}$

In [ ]:
x = np.arange(-1, 1, 0.05)
def formula(x): return x**4 + x**3 + x**2 + x 
y = list(map(formula, x))

In [ ]:
import matplotlib.pyplot as plt 
fig = plt.figure(figsize = (4,3))
ax = fig.add_subplot(111)
ax.plot(x, y, 'bo-', markersize = 3, linewidth = 1)
plt.show()

<b>0) Primitive definition </b>

In [ ]:
# define primtives
arity_main = 1 # only x
pset = gp.PrimitiveSet('main', 1) # contain a set of primitives to be used
pset.addPrimitive(np.add, 2, name = "add") # operator, arity = the number of operands, name 
pset.addPrimitive(np.multiply, 2, name = "multiply")
pset.addPrimitive(np.subtract, 2, name = "substract")
pset.addPrimitive(np.negative, 1, name = "neg")
pset.renameArguments(ARG0 = "x")
#pset.addTerminal(1)

<b>1) Type defintion </b>

In [ ]:
creator.create("SRFitnessMin", base.Fitness, weights = (-1.0,))
creator.create("SRIndividual", gp.PrimitiveTree, fitness = creator.SRFitnessMin, pset = pset)

<b>2) Initialisation </b>

In [ ]:
min_depth = 0
init_max_depth = 3 
pop_size = 50

sr_toolbox = base.Toolbox()
# gp.genFull, gp.genHalfAndHalf, gp.genGrow
sr_toolbox.register("expr", gp.genFull, pset = pset, min_ = min_depth, max_ = init_max_depth)
sr_toolbox.register("individual", tools.initIterate, creator.SRIndividual, sr_toolbox.expr)
sr_toolbox.register("population", tools.initRepeat, list, sr_toolbox.individual, n = pop_size)

<b>3) Operator defintion & register </b>

In [ ]:
def eval_expr(expr, x, y, pset):
    from sklearn.metrics import mean_squared_error
    # Y = the ground truth 
    comp_y = gp.compile(expr, pset)(x)
    mse = mean_squared_error(y, comp_y, squared=False)
    return (mse,)  

sr_toolbox.register("evaluate", eval_expr, x = x, y = y, pset = pset)
sr_toolbox.register("mate", gp.cxOnePoint)
sr_toolbox.register("mutate", gp.mutUniform, expr = sr_toolbox.expr, pset = pset)
sr_toolbox.register("select", tools.selTournament, tournsize = 3)

In [ ]:
# limit
import operator 
max_tree_depth = 20
sr_toolbox.decorate("mate", 
    gp.staticLimit(
        key = operator.attrgetter("height"), 
        max_value = max_tree_depth
    )
)

sr_toolbox.decorate("mutaet", 
    gp.staticLimit(
        key = operator.attrgetter("height"), 
        max_value = max_tree_depth
    )
)

<b>4) Algorithm </b>

In [ ]:
n_gen = 20
cxRate = 1.0
mutRate = 0.2

In [ ]:
# 1) population initialisaiton 
pop = sr_toolbox.population(n = pop_size)

In [ ]:
# 2) fitness evaluation 
fitness_per_ind = sr_toolbox.map(sr_toolbox.evaluate, pop) 
for ind,fit_v in zip(pop, fitness_per_ind):
    ind.fitness.values = fit_v 

In [ ]:
# 2-2) select the best individual & store
best = tools.HallOfFame(1)
best.update(pop)
print (best[0], best[0].fitness.values)

In [ ]:
# logging 
stats = tools.Statistics(lambda ind:ind.fitness.values[0])
stats.register("max", np.max, axis = 0)
stats.register("min", np.min, axis = 0)
stats.register("mean",np.mean, axis = 0)
stat_results = stats.compile(pop)
logbook = tools.Logbook()
logbook.record(gen = 0, min = stat_results['min'], max = stat_results['max'], mean = stat_results['mean'])
print (logbook)

In [ ]:
# 3) evolution & 4) next population selection 
for gen_idx in range(1, n_gen + 1):
    # 3) evolution
    offsprings = []
    pop_copied = [sr_toolbox.clone(ind) for ind in pop]
    while len(offsprings) < pop_size:
        offspring = sr_toolbox.select(pop_copied, 2)
        ind1, ind2 = algorithms.varAnd(offspring, sr_toolbox, cxRate, mutRate)
        offsprings.extend([ind1, ind2])
    
    ind_fit_inv = [ind for ind in offsprings if not ind.fitness.valid]
    fitness_per_ind = sr_toolbox.map(sr_toolbox.evaluate, ind_fit_inv)
    for ind, fit_v in zip(ind_fit_inv, fitness_per_ind):
        ind.fitness.values = fit_v 
    
    # 4) next popluation selection 
    pop[:] = tools.selBest(pop + offsprings, pop_size)
    
    # update the best
    best.update(pop)

    # logging
    stat_results = stats.compile(pop)
    logbook.record(gen = gen_idx, 
        min = stat_results['min'], max = stat_results['max'], mean = stat_results['mean']
    )
    print (logbook)
    print (f"\tGen {gen_idx}:\n\t\tbest:{str(best[0])}\n\t\tfitness:{best[0].fitness.values[0]}")

In [ ]:
str(best[0])

In [ ]:
y_computed = gp.compile(best[0], pset=pset)(x)

import matplotlib.pyplot as plt 
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.plot(x, y, 'bo-', markersize = 3, linewidth = 1)
ax.plot(x, y_computed, 'ro-', markersize = 3, linewidth = 1)
ax.plot(x, y - y_computed, 'go-', markersize = 3, linewidth = 1)
plt.show()

In [ ]:
#out = algorithms.eaSimple(pop, sr_toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=False)

#### Exercises

<b>Even Parity Generator</b>

* Individual: a candidate expression/generator 
* Fitness: 
    - Maximise the number of correctly computed even parity bits 
    - Or Minimise the number of wrongly computed even parity bits
* crossover: a single point crossover 
    - gp.cxOnepoint 
* mutation: 
    - gp.mutUniform
* parent selection
    - tools.selTournament, tournsize = 3
* next population selection: select the top pop_size best individuals 
    - tools.selBest 
* primitives:
    - operator.and_, operator.or_, operator.xor, operator.not_
* terminals:
    - 1 and 0 

In [ ]:
import operator 

In [ ]:
pop_size = 100 
seq_size = 6
n_parity = 2**seq_size

In [ ]:
# provided
# dataset
data_seqs = []
parity_bits =[]
for i in range(n_parity):
    data_seqs.append([])
    v = i 
    parity = 0 
    dividor = n_parity
    for j in range(seq_size):
        dividor /= 2
        if v >= dividor:
            data_seqs[-1].append(1)
            parity = int(not parity)
            v -= dividor 
        else:
            data_seqs[-1].append(0)
    parity_bits.append(parity)

In [ ]:
import operator 

# define a primtive set
pset = ... # name = "main" 
#perator.and_, operator.or_, operator.xor, operator.not_
...

# define types 
# e.g., EPFitnessMax, EPIndividual

# initialisation 
min_depth = 3
init_max_depth = 5
# e.g., expr => a tree generator/initialiser, individual, population 
...

# operators
def eval_generator(expr, x, y, pset):
    ...

# evaluate, mate (crossover), mutate, select 

n_gen = 50
cxRate = 0.5
mutRate = 0.2

# 1) population initialisaiton 
...
# 2) fitness evaluation 
...
# 2-2) select the best individual & store
...

# logging 
#stats = tools.Statistics(lambda ind:ind.fitness.values[0])
#stats.register("max", np.max, axis = 0)
#stats.register("min", np.min, axis = 0)
#stats.register("mean",np.mean, axis = 0)
#stat_results = stats.compile(pop)
#logbook = tools.Logbook()
#logbook.record(
    #gen = 0, min = stat_results['min'], max = stat_results['max'], mean = stat_results['mean'])
#print (logbook)

# 3) evolution & 4) next population selection 
for gen_idx in range(1, n_gen + 1):
    # 3) evolution
    ...
    
    # 4) next population selection & fitness computation for new offsrpings 
    # fitness computation for offsrpings
    ...
    
    # the next population selection 
    ...
    # update the best for next pop 
    ...

    # logging
    #stat_results = stats.compile(pop)
    #logbook.record(gen = gen_idx, 
        #min = stat_results['min'], max = stat_results['max'], mean = stat_results['mean']
    #)
    #print (logbook)
    #print (f"\tGen {gen_idx}:\n\t\tbest:{str(best[0])}\n\t\tfitness:{best[0].fitness.values[0]}")